In [5]:
import numpy as np
from ase.io import read, write
from numpy.linalg import norm
from numpy import (dot, arccos, clip)
import math
from sympy import Symbol, nsolve
import sympy
import mpmath


In [ ]:
#function to compute lattice angles 
def angleF (v1, v2) :
    cosAngle = np.dot(v1,v2)/norm(v1)/norm(v2)
    angle = math.acos(np.clip(cosAngle, -1, 1))
    return angle

#function to symmetrize the lattice in the following way
# [[a1 a2 a2]
#  [b1 b2 b1]
#  [c1 c1 c3]]
#other schemes should be relavent too, for example ; this will be implemented later
# [[a1 a2 a3]
#  [a2 b2 b3]
#  [a3 b3 c3]]
def symmetrizeLattice (lat) :
    #computing lattice parameters
    a     = math.sqrt(lat[0][0]**2+lat[0][1]**2+lat[0][2]**2)
    b     = math.sqrt(lat[1][0]**2+lat[1][1]**2+lat[1][2]**2)
    c     = math.sqrt(lat[2][0]**2+lat[2][1]**2+lat[2][2]**2)
    gamma = angleF (lat[0],lat[1])
    alpha = angleF (lat[1],lat[2])
    beta  = angleF (lat[0],lat[2])
    #print(a, b, c, alpha*180/math.pi, beta*180/math.pi, gamma*180/math.pi)
    a11 = Symbol('a11')
    a12 = Symbol('a12')
    b21 = Symbol('b21')
    b22 = Symbol('b22')
    c31 = Symbol('c31')
    c33 = Symbol('c33')

    asym     = (a11/a)**2  + (a12/a)**2  + (a12/a)**2  - 1
    bsym     = (b21/b)**2  + (b22/b)**2  + (b21/b)**2  - 1
    csym     = (c31/c)**2  + (c31/c)**2  + (c33/c)**2  - 1

    alphasym = a11*b21/(a*b) + a12*b22/(a*b) + a12*b21/(a*b) - math.cos(gamma)
    betasym  = b21*c31/(b*c) + b22*c31/(b*c) + b21*c33/(b*c) - math.cos(alpha)
    gammasym = a11*c31/(a*c) + a12*c31/(a*c) + a12*c33/(a*c) - math.cos(beta)
    sol=nsolve(\
               (asym, bsym, csym, alphasym, betasym, gammasym),\
               (a11, a12, b21, b22, c31, c33),\
               (1, 0, 0, 1, 0, 1)\
              )

    A=[float(sol[0]),float(sol[1]),float(sol[1])] # [[a1 a2 a2]
    B=[float(sol[2]),float(sol[3]),float(sol[2])] #  [b1 b2 b1]
    C=[float(sol[4]),float(sol[4]),float(sol[5])] #  [c1 c1 c3]]
    rotlat=np.matmul([A,B,C],[[1,0,0],[0,1,0],[0,0,1]])    #casting the lattice vectors into matrix form
    #test printing the rotated lattice constants and their differences with input lattice
    #print(norm(A), norm(B), norm(C), angleF(B,C)*180/math.pi, angleF(A,C)*180/math.pi, angleF(A,B)*180/math.pi )
    #print(norm(A)-a, norm(B)-b, norm(C)-c, angleF(B,C)-alpha, angleF(A,C)-beta, angleF(A,B)-gamma )

    return rotlat #has to be changed

In [ ]:
relaxed=read('/pscratch/sd/k/kcpitike/MPEA/ternaries/Cr-Ti-W/1/11/CONTCAR')
relaxed.write('/pscratch/sd/k/kcpitike/MPEA/ternaries/Cr-Ti-W/1/11/POSCAR_unsymmetrized',direct=1)
lat=relaxed.get_cell()
symLat    = symmetrizeLattice(lat)

print(math.sqrt(lat[0][0]**2+lat[0][1]**2+lat[0][2]**2) - math.sqrt(symLat[0][0]**2+symLat[0][1]**2+symLat[0][2]**2))
print(math.sqrt(lat[1][0]**2+lat[1][1]**2+lat[1][2]**2) - math.sqrt(symLat[1][0]**2+symLat[1][1]**2+symLat[1][2]**2))
print(math.sqrt(lat[2][0]**2+lat[2][1]**2+lat[2][2]**2) - math.sqrt(symLat[2][0]**2+symLat[2][1]**2+symLat[2][2]**2))

print(angleF(lat[0],lat[1])-angleF(symLat[0],symLat[1]))
print(angleF(lat[1],lat[2])-angleF(symLat[1],symLat[2]))
print(angleF(lat[0],lat[2])-angleF(symLat[0],symLat[2]))

relaxed=read('/pscratch/sd/k/kcpitike/MPEA/ternaries/Cr-Ti-W/1/11/CONTCAR')
strained=read('/pscratch/sd/k/kcpitike/MPEA/ternaries/Cr-Ti-W/1/11/CONTCAR')
lat=relaxed.get_cell()[:]
symLat    = symmetrizeLattice(lat)
coords=relaxed.get_scaled_positions()

strained.set_cell(symLat)
strained.set_scaled_positions(coords)
strained.write('/pscratch/sd/k/kcpitike/MPEA/ternaries/Cr-Ti-W/1/11/POSCAR_symmetrized',direct=1)

for i in range(0,6) :                        #i is the strain component index
    for st in [-0.01,0.01] :                 #st is the strain state ; example, -0.01 = -1% etc.
        relaxed=read('/pscratch/sd/k/kcpitike/MPEA/ternaries/Cr-Ti-W/1/11/CONTCAR')
        strained=read('/pscratch/sd/k/kcpitike/MPEA/ternaries/Cr-Ti-W/1/11/CONTCAR')
        lat=relaxed.get_cell()[:]
        symLat    = symmetrizeLattice(lat)
        coords=relaxed.get_scaled_positions()
        strainLat = symLat[:]                  #initiating a strained lattice
        if (i==0 or i==1 or i==2) :
            #print(i,st)
            strainLat[0,i] =   strainLat[0,i] * (1+st)
            strainLat[1,i] =   strainLat[1,i] * (1+st)
            strainLat[2,i] =   strainLat[2,i] * (1+st)
        if (i==3) :
            #print(i,st)
            strainLat[1,2] =   np.linalg.norm(strainLat[1]) * (st/2) + strainLat[1,2]
            strainLat[2,1] =   np.linalg.norm(strainLat[2]) * (st/2) + strainLat[2,1]
        if (i==4) :
            #print(i,st)
            strainLat[0,2] =   np.linalg.norm(strainLat[0]) * (st/2) + strainLat[0,2]
            strainLat[2,0] =   np.linalg.norm(strainLat[2]) * (st/2) + strainLat[2,0]
        if (i==5) :
            #print(i,st)
            strainLat[0,1] =   np.linalg.norm(strainLat[0]) * (st/2) + strainLat[0,1]
            strainLat[1,0] =   np.linalg.norm(strainLat[1]) * (st/2) + strainLat[1,0]
        print("Done computing strained lattice in", i+1, "direction and ", st*100, "% strain")
        #print(strainLat)
        outFileName = '/pscratch/sd/k/kcpitike/MPEA/ternaries/Cr-Ti-W/1/11/POSCAR.%02d.%02d'%(i+1, int(st*100))
        print("Writing",outFileName)
        strained.set_cell(strainLat)
        strained.set_scaled_positions(coords)
        strained.write(outFileName,direct=1)

0.0
0.0
0.0
0.0
0.0
0.0
[[ 9.23807473e+00 -3.71519189e-02 -3.71519189e-02]
 [ 2.65368471e-02  9.22564922e+00  2.65368471e-02]
 [ 2.41101726e-04  2.41101726e-04  9.20798794e+00]]
Done computing strained lattice in 1 direction and  -1.0 % strain
Writing /pscratch/sd/k/kcpitike/MPEA/ternaries/Cr-Ti-W/1/11/POSCAR.01.-1
[[ 9.14569398e+00 -3.71519189e-02 -3.71519189e-02]
 [ 2.62714786e-02  9.22564922e+00  2.65368471e-02]
 [ 2.38690709e-04  2.41101726e-04  9.20798794e+00]]
Done computing strained lattice in 1 direction and  1.0 % strain
Writing /pscratch/sd/k/kcpitike/MPEA/ternaries/Cr-Ti-W/1/11/POSCAR.01.01
[[ 9.23715092e+00 -3.71519189e-02 -3.71519189e-02]
 [ 2.65341934e-02  9.22564922e+00  2.65368471e-02]
 [ 2.41077616e-04  2.41101726e-04  9.20798794e+00]]
Done computing strained lattice in 2 direction and  -1.0 % strain
Writing /pscratch/sd/k/kcpitike/MPEA/ternaries/Cr-Ti-W/1/11/POSCAR.02.-1
[[ 9.23715092e+00 -3.67803997e-02 -3.71519189e-02]
 [ 2.65341934e-02  9.13339273e+00  2.65368471e-